새로운 고객에 대한 분류 예측

11개의 범주로 구분되어 있으며, 그중 Segmentation을 종속변수로 나머지 변수를 독립변수로 설정하여 분석을 진행함.

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

df1 = pd.read_csv('/content/drive/MyDrive/데이터 셋/P220404-01.csv')
print(df1.info())
print(df1.describe())
print(df1.head())


df2 = pd.read_csv('/content/drive/MyDrive/데이터 셋/P220404-02.csv')

print(df2.info())
print(df2.describe())
print(df2.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               6665 non-null   int64 
 1   Gender           6665 non-null   object
 2   Ever_Married     6665 non-null   object
 3   Age              6665 non-null   int64 
 4   Graduated        6665 non-null   object
 5   Profession       6665 non-null   object
 6   Work_Experience  6665 non-null   int64 
 7   Spending_Score   6665 non-null   object
 8   Family_Size      6665 non-null   int64 
 9   Var_1            6665 non-null   object
 10  Segmentation     6665 non-null   object
dtypes: int64(4), object(7)
memory usage: 572.9+ KB
None
                 ID          Age  Work_Experience  Family_Size
count    6665.00000  6665.000000      6665.000000  6665.000000
mean   463519.84096    43.536084         2.629107     2.841110
std      2566.43174    16.524054         3.405365     1.524743

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

le = LabelEncoder()
label = ['Profession']
df1[label] = le.fit_transform(df1[label]).reshape(-1,1)
df2[label] = le.fit_transform(df2[label]).reshape(-1,1)

df1 = pd.get_dummies(df1, columns = ['Gender', 'Ever_Married','Graduated','Spending_Score','Var_1'])
df2 = pd.get_dummies(df2, columns = ['Gender', 'Ever_Married','Graduated','Spending_Score','Var_1'])

mms = MinMaxScaler()
numbers = ['Age','Work_Experience','Family_Size']
df1[numbers] = mms.fit_transform(df1[numbers])
df2[numbers] = mms.fit_transform(df2[numbers])

print(df1)
print(df2)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


          ID       Age  Profession  Work_Experience  Family_Size Segmentation  \
0     462809  0.056338           5         0.071429        0.375            D   
1     466315  0.690141           2         0.071429        0.000            B   
2     461735  0.690141           7         0.000000        0.125            B   
3     461319  0.535211           0         0.000000        0.125            C   
4     460156  0.197183           5         0.071429        0.250            C   
...      ...       ...         ...              ...          ...          ...   
6660  463002  0.323944           0         0.000000        0.500            B   
6661  464685  0.239437           4         0.214286        0.375            D   
6662  465406  0.211268           5         0.071429        0.000            D   
6663  467299  0.126761           5         0.071429        0.375            B   
6664  461879  0.267606           4         0.000000        0.250            B   

      Gender_Female  Gender

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

df1['Segmentation'] = df1['Segmentation'].astype('category')

x = df1.drop('Segmentation', axis=1)
y = df1['Segmentation']

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2,random_state=42)

md = RandomForestClassifier(n_estimators=50, max_depth=6)
md.fit(x_train.drop('ID', axis=1), y_train)

pred = md.predict(x_valid.drop('ID',axis=1))

cm = confusion_matrix(y_valid, pred, labels=['A','B','C','D'])
print('혼동행렬: ', cm)
print('정확도: ', accuracy_score(y_valid, pred))

print('f1-스코어: ', f1_score(y_valid, pred, average='macro'))

혼동행렬:  [[163  44  43  65]
 [ 90  72 107  33]
 [ 46  44 207  49]
 [ 93  13  14 250]]
정확도:  0.5191297824456114
f1 스코어:  0.49838937201471145


모델에 대한 정확도는 0.51, f1-스코어는 0.50으로 좋은 성능은 보이지 못하며 추후 추가적인 과정이 필요할 것으로 보인다




In [ ]:
x_test = df2

md.fit(x.drop('ID',axis=1), y)

pred = md.predict(x_test.drop('ID',axis=1))
print(pred)

result = pd.DataFrame({'ID': df2['ID'], 'pred': pred})
print(result)

구축된 모델에 테스트 데이터를 적용시켜 새로운 고객에 대한 분류를 예측해 보았다.